In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
# Make sure we're in the right directory
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

## Inference with ClimART data and models

In [8]:
import sys
import numpy as np
import pytorch_lightning as pl
from climart.utils.config_utils import get_config_from_hydra_compose_overrides
from climart.interface import get_model_and_data

In [18]:
num_workers=2

In [19]:
os.environ['WANDB_SILENT']="true"
np.set_printoptions(suppress=True, threshold=sys.maxsize)

In [22]:
exp_type='pristine'
predict_years = "2012-14"
overrides = [f'model=mlp',
             f'datamodule.exp_type={exp_type}',
             f'datamodule.num_workers={num_workers}', 
             f'++datamodule.predict_years={predict_years}']
cfg = get_config_from_hydra_compose_overrides(overrides)
model, dm = get_model_and_data(cfg)
trainer = pl.Trainer(gpus=-1)

Multiprocessing is handled by SLURM.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [55]:
results1 = trainer.predict(model=model,datamodule=dm, return_predictions=True)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [62]:
results1 = model.aggregate_predictions(results1)
sw_hr_preds_2014 = results1[2014]['preds']['hrsc']
sw_hr_preds_2014.shape

(122880, 49)

#### Alternatively to command line/config-directed changing of the predict_years, you can also do this programmatically to an existing datamodule as follows:

In [79]:
dm.predict_years = "2010"
results2 = trainer.predict(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [80]:
results2[0]['preds']['rsuc'].shape

torch.Size([512, 50])

In [81]:
results2 = model.aggregate_predictions(results2)
sw_hr_preds_2010 = results2[2010]['preds']['hrsc']
sw_hr_preds_2010.shape

(122880, 49)